## Synopsis

This script takes the averaged HDF files on `/net/wolf` with G2, removes the hot pixels in first delay point of G2, and saves into a regular size and format HDF result file with cleaned up g2


## Initialization

In [1]:

import numpy as np
from numpy import sqrt, sin, cos, pi, exp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import h5py  
from subprocess import PIPE, run
from xpcs_viewer import XpcsFile as xf

# Local file `XPCS_Functions.py`
from XPCS_Functions import G2_g2


## List target file names 

In [2]:

fp_remote = '/net/wolf/data/xpcs8/2020-3/qzhang202010/cluster_results_XPCS/'

fn_remote = ('J058/AvgJ058_00001_CPMV_6C_7v1_Water_att00_Try1_0001-100000.hdf',
             'J058_1st50k/AvgJ058_00001_CPMV_6C_7v1_Water_att00_Try1_0001-100000.hdf',
             'J058_2nd50k/AvgJ058_00001_CPMV_6C_7v1_Water_att00_Try1_0001-100000.hdf',
             'M071_M073/AvgM071_00001_CPMV_6C_7v1_NH4M_att00_Try1_0001-100000.hdf',
             'L067/AvgL067_00001_CPMV_6C_7v1_NaCl4M_att00_Try1_0001-100000.hdf')

fn_local = ('J058_Water_XPCS_Trim.hdf',
            'J058_Water_1st_XPCS_Trim.hdf',
            'J058_Water_2nd_XPCS_Trim.hdf',
            'M071_NH4_XPCS_Trim.hdf',
            'L067_NaCl_XPCS_Trim.hdf')
                

In [3]:

for ii in range(5):
    a = xf(fn_remote[ii], cwd = fp_remote, fields=['G2','IP','IF','sqmap','dqmap'])
    
    # Bin G2 into g2
    [a.g2_fix,a.g2_err_fix] = G2_g2(a.G2,a.IP,a.IF,a.sqmap,a.dqmap)

    # Copy hdf5 result file with G2 into a tmp file at current directory
    command = ['cp', fp_remote+fn_remote[ii], 'tmp.hdf']
    _ = run(command, stdout=PIPE, stderr=PIPE)

    # # Remove G2/IP/IF from the tmp file using h5repack
    with h5py.File('tmp.hdf', 'r+') as f1:
        del f1['/exchange/G2']
        del f1['/exchange/IP']
        del f1['/exchange/IF']
        f1['/exchange/norm-0-g2'][...] = a.g2_fix
        f1['/exchange/norm-0-stderr'][...] = a.g2_err_fix
        f1.flush()
               
    # Copy tmp file to another file with the specified file name
    command = ['h5repack', 'tmp.hdf', fn_local[ii]]
    _ = run(command, stdout=PIPE, stderr=PIPE)
    
# Delete the tmp file
command = ['rm', '-rf', 'tmp.hdf']
_ = run(command, stdout=PIPE, stderr=PIPE)

